##### modeling and interpretation notebook--train, test, and conclude
***

# Importing Libraries and Data

In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Import Model Specific Libraries

In [2]:
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
import statsmodels.stats.api as sms
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

import math

### Reopening our cleaned dataframe

In [3]:
pd.set_option('display.max_columns', 999)  # setting to view all columns

with open('data/no_hot_drops.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    no_hot_drops_df = pickle.load(f)
display(no_hot_drops_df.shape)
print("\nDataframe successfully imported from pickle.\n\n", no_hot_drops_df.head())

(20533, 18)


Dataframe successfully imported from pickle.

            id  date     price  bedrooms  bathrooms  sqft_living  sqft_lot  \
0  7129300520  2014  221900.0         3       1.00         1180      5650   
1  6414100192  2014  538000.0         3       2.25         2570      7242   
2  5631500400  2015  180000.0         2       1.00          770     10000   
3  2487200875  2014  604000.0         4       3.00         1960      5000   
4  1954400510  2015  510000.0         3       2.00         1680      8080   

   floors  view  grade  sqft_above  sqft_basement zipcode      lat     long  \
0     1.0     0      7        1180            0.0   98178  47.5112 -122.257   
1     2.0     0      7        2170          400.0   98125  47.7210 -122.319   
2     1.0     0      6         770            0.0   98028  47.7379 -122.233   
3     1.0     0      7        1050          910.0   98136  47.5208 -122.393   
4     1.0     0      8        1680            0.0   98074  47.6168 -122.045   

   sqft_living

# Questions

We can begin investigating our remaining questions by building and testing a model, selecting and eliminating features, and adjusting variable coefficients along the way. Remember, we started by looking for strong correlations between our target (price) and independent variables in the King County housing data set. We found that some predictors could be eliminated, due to their lack of correlation or linearity with the response variable. We also found that some variables shoulds be eliminated from our study, due to their outsized effect on others. We will see more of this, as we look at collinearity, p-values, et al., in this notebook.

As a reminder of our remaining basic questions:

2. Which variables (whether redundant or un - influential) can or should be eliminated toward building a predictive model for home price?
3. Can we make a reasonably compelling model without categorical data?
4. Does model effectiveness vary across zip codes?

## Correlations

### Seeing where we stand

In [4]:
# create the correlations table
nh_corr = abs(no_hot_drops_df.corr())

# view the table--though we will not study it too closely
nh_corr

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,view,grade,sqft_above,sqft_basement,lat,long,sqft_living15,sqft_lot15,Renovated
id,1.000000,0.010051,0.008117,0.010506,0.025506,0.001689,0.130250,0.027060,0.016306,0.022300,0.006637,0.007424,0.000799,0.025777,0.007257,0.143096,0.009978
date,0.010051,1.000000,0.005013,0.010222,0.028198,0.032341,0.003108,0.023734,0.000009,0.034535,0.027574,0.011527,0.030099,0.000087,0.022972,0.002050,0.020872
price,0.008117,0.005013,1.000000,0.277597,0.440857,0.639054,0.075040,0.239718,0.369763,0.646090,0.531310,0.263419,0.357851,0.014904,0.575582,0.066471,0.121323
bedrooms,0.010506,0.010222,0.277597,1.000000,0.480560,0.582757,0.024973,0.159189,0.051023,0.337962,0.472042,0.259799,0.032828,0.156122,0.404916,0.024987,0.003735
bathrooms,0.025506,0.028198,0.440857,0.480560,1.000000,0.707837,0.058401,0.508427,0.132606,0.628571,0.630299,0.216807,0.004162,0.240263,0.539457,0.059269,0.029649
sqft_living,0.001689,0.032341,0.639054,0.582757,0.707837,1.000000,0.147089,0.338085,0.232098,0.733436,0.852384,0.374269,0.038599,0.256960,0.756984,0.162351,0.043577
sqft_lot,0.130250,0.003108,0.075040,0.024973,0.058401,0.147089,1.000000,0.019890,0.070629,0.091734,0.157995,0.001938,0.093254,0.220081,0.135656,0.698301,0.005793
floors,0.027060,0.023734,0.239718,0.159189,0.508427,0.338085,0.019890,1.000000,0.000845,0.447654,0.526310,0.289531,0.043815,0.123647,0.260577,0.028895,0.002810
view,0.016306,0.000009,0.369763,0.051023,0.132606,0.232098,0.070629,0.000845,1.000000,0.209300,0.109146,0.240825,0.000683,0.087008,0.248507,0.069418,0.089739
grade,0.022300,0.034535,0.646090,0.337962,0.628571,0.733436,0.091734,0.447654,0.209300,1.000000,0.724873,0.100708,0.110384,0.196205,0.690127,0.099243,0.010519


I printed the correlations table for a couple of reasons:
* First, I want to confess that I have been hanging onto that `date` field, but I have yet to find a good use for it. We'll drop it, following the next visualization.
* Secondly, I want to point out how the dataframe can be re-ordered to be more useful, visually. We can order the columns by the strength of their correlation with price, so that is what we will do.

### Viewing columns sorted in order of correlation with price

In [9]:
# running a sorted correlation and grabbing indexes
sort_ix = abs(
    nh_corr).sort_values(
    'price', ascending=False).index
df_sortix = no_hot_drops_df.loc[:, sort_ix]


df_sortix

,price,grade,sqft_living,sqft_living15,sqft_above,bathrooms,view,lat,bedrooms,sqft_basement,floors,Renovated,sqft_lot,sqft_lot15,long,id,date
0,221900.0,7,1180,1340,1180,1.00,0,47.5112,3,0.0,1.0,0,5650,5650,-122.257,7129300520,2014
1,538000.0,7,2570,1690,2170,2.25,0,47.7210,3,400.0,2.0,1,7242,7639,-122.319,6414100192,2014
2,180000.0,6,770,2720,770,1.00,0,47.7379,2,0.0,1.0,0,10000,8062,-122.233,5631500400,2015
3,604000.0,7,1960,1360,1050,3.00,0,47.5208,4,910.0,1.0,0,5000,5000,-122.393,2487200875,2014
4,510000.0,8,1680,1800,1680,2.00,0,47.6168,3,0.0,1.0,0,8080,7503,-122.045,1954400510,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,360000.0,8,1530,1530,1530,2.50,0,47.6993,3,0.0,3.0,0,1131,1509,-122.346,263000018,2014
21593,400000.0,8,2310,1830,2310,2.50,0,47.5107,4,0.0,2.0,0,5813,7200,-122.362,6600060120,2015
21594,402101.0,7,1020,1020,1020,0.75,0,47.5944,2,0.0,2.0,0,1350,2007,-122.299,1523300141,2014
21595,400000.0,8,1600,1410,1600,2.50,0,47.5345,3,0.0,2.0,0,2388,1287,-122.069,291310100,2015


## Answer 2.4
### Which variables can or should be eliminated
We now see the date column, appropriately distanced from our target (and the correlation value that makes it so). Let's remove it and view the heatmap.

### Drop `date`

In [16]:
# drop the `date` column
df_sortix.drop(['date'], axis=1, inplace=True)

### Viewing the sorted correlations table

In [17]:
# view without `id`
df_sortix.drop(['id'], axis=1).corr()

,price,grade,sqft_living,sqft_living15,sqft_above,bathrooms,view,lat,bedrooms,sqft_basement,floors,Renovated,sqft_lot,sqft_lot15,long
price,1.000000,0.646090,0.639054,0.575582,0.531310,0.440857,0.369763,0.357851,0.277597,0.263419,0.239718,0.121323,0.075040,0.066471,0.014904
grade,0.646090,1.000000,0.733436,0.690127,0.724873,0.628571,0.209300,0.110384,0.337962,0.100708,0.447654,0.010519,0.091734,0.099243,0.196205
sqft_living,0.639054,0.733436,1.000000,0.756984,0.852384,0.707837,0.232098,0.038599,0.582757,0.374269,0.338085,0.043577,0.147089,0.162351,0.256960
sqft_living15,0.575582,0.690127,0.756984,1.000000,0.722971,0.539457,0.248507,0.041955,0.404916,0.149249,0.260577,-0.003066,0.135656,0.175432,0.337272
sqft_above,0.531310,0.724873,0.852384,0.722971,1.000000,0.630299,0.109146,-0.021357,0.472042,-0.157598,0.526310,0.012606,0.157995,0.173110,0.366948
bathrooms,0.440857,0.628571,0.707837,0.539457,0.630299,1.000000,0.132606,0.004162,0.480560,0.216807,0.508427,0.029649,0.058401,0.059269,0.240263
view,0.369763,0.209300,0.232098,0.248507,0.109146,0.132606,1.000000,0.000683,0.051023,0.240825,0.000845,0.089739,0.070629,0.069418,-0.087008
lat,0.357851,0.110384,0.038599,0.041955,-0.021357,0.004162,0.000683,1.000000,-0.032828,0.109864,0.043815,0.028170,-0.093254,-0.098811,-0.137848
bedrooms,0.277597,0.337962,0.582757,0.404916,0.472042,0.480560,0.051023,-0.032828,1.000000,0.259799,0.159189,0.003735,0.024973,0.024987,0.156122
sqft_basement,0.263419,0.100708,0.374269,0.149249,-0.157598,0.216807,0.240825,0.109864,0.259799,1.000000,-0.289531,0.058432,-0.001938,-0.001234,-0.161391


In [13]:
# create a function to generate a heatmap
def plot_corr_heatmap(cor, cmap="BuPu"):
    # creating a figure to plot a heatmap from correlations
    plt.figure(figsize=(21,18))
    sns.set(font_scale=1.5, color_codes=True) # making annotations legible
    # plotting
    sns.heatmap(cor.round(2).abs(), center=.4, annot=True, cmap="vlag")
    plt.xticks(rotation = 70, fontsize=20)
    plt.yticks(fontsize=20)
    plt.autoscale() # prevent cropping
    plt.show()

In [ ]:
plot_corr_heatmap(df_sortix.drop('id', axis=1))

## Answer 2.5
### Which variables can or should be eliminated

Without much difficulty, we can see a white block in the first column indicating that `sqft_lot15` is not highly corellated with price. Further, we see a darker red block in the `sqft_lot15` row, indicating that the feature is highly coordinated with `sqft_above`. Of the two, `sqft_lot15` has the weaker relationship with `price`, so that is the one we can eliminate.

So, let's.

In [ ]:
# drop the `sqft_lot15` column
no_hot_drops_df.drop('sqft_lot15', axis=1, inplace=True)

There are several more weak target correlations and strong predictor correlations. We can use our python libraries and modules to work them out.

First, let's pickle our dataframe in its current state and check our distributions.

In [ ]:
with open('data/no-hot-drops-df.pickle', 'wb') as f:
    # Pickling the 'no_hots_df' dataframe using the highest protocol available.
    pickle.dump(no_hot_drops_df, f, pickle.HIGHEST_PROTOCOL)

Now, we can do another check to see whether our independent variable distributions are approximately normal and, if not, to take appropriate steps. This will make our modeling predictions more reliable.

## Checking Normality

In [ ]:
# copying our dataframe to a new `normal_df`
normal_df = no_hot_drops_df.copy()

normal_df.columns

### Raw Features Model

Let's set a baseline, with features as they are. Then we will make any transformations that appear promising and see whether they improve the model.

In [ ]:
# set outcome ('target')
outcome = 'price'
# initialize variable for columns to submit to ols
x_cols = ['grade', 'sqft_living', 'sqft_living15', 'sqft_above', 'bathrooms', 'bedrooms','sqft_basement', 'Renovated', 'sqft_lot']
# define the formula to run the inputs
predictors = '+'.join(x_cols)
formula = outcome + '~' + predictors

# name the model request and compute a summary
model = ols(formula=formula, data=normal_df).fit()
model.summary()

We are skipping analysis and moving to transformation, so we can create a comparison model and validate improvement. Let's review the current shape univariate distributions for features in our data set.

In [ ]:
# viewing a pairplot for regression model columns
sns.pairplot(normal_df[x_cols], kind="reg", plot_kws={'line_kws':{'color':'orange'}})
sns.set(font_scale=.8) # try to prevent overlap of long column names
plt.subplots_adjust(wspace=.01, hspace=.1);
plt.show()

I think it only makes sense to focus on the `sqft` features for log transformation. If necessary, we can come back to categorical values, after we have a model. Remember, one of our questions is whether we can build a sufficient model without using categorical features. So, we'll see what only transforming continuous features does for us.

Those zero values in `sqft_basement` will not work, though

In [ ]:
# list the continuous variable column names
non_normal = ['sqft_living', 'sqft_living15', 'sqft_above','sqft_basement', 'sqft_lot']

# transform columns data in a loop using a lambda function
for feat in non_normal:
    normal_df[feat] = normal_df[feat].map(lambda x: np.log(x))

A zero encountered in log: I bet it's in the basement.

### More Cleaning

In [ ]:
for feat in non_normal:
    print(f"{feat}: {normal_df[feat].min()}")
    
display(normal_df.dtypes)

Looks like `sqft_basement` did not handle that normalization so well and now contains Nan values.

In [ ]:
normal_df[normal_df.sqft_basement.isna()]

In [ ]:
for col in normal_df.columns:
    if normal_df[col].dtype != np.int32:
        print(f"{col} has {normal_df[col].isnull().count().count()} null values")
# normal_df.isnull().count().count()

Let's see what shape we are in, now.

In [ ]:
# viewing a scatter matrix with log - transformed regression model columns

sns.pairplot(normal_df[x_cols], kind="reg", plot_kws={'line_kws':{'color':'orange'}})
sns.set(font_scale=.8) # try to prevent overlap of long column names
plt.subplots_adjust(wspace=.01, hspace=.1);
plt.show()

## OLS and Collinearity

In [ ]:
# plt.figure(figsize=(8,4))
# plt.hist(target, color='orange')
# plt.show()

In [ ]:
# viewing pairwise relationships
# sns.pairplot(data_f)

# plt.show()

The pairplots give a feel for just how much of the data is categorical.

## Jarque - Bera and Kurtosis
## Questions and Desired Variables
## Model & Testing
## Conclusion

In [ ]:
# viewing pairwise relationships
# sns.pairplot(no_hot_drops)
# plt.show()

## Answer 1.2
### Strongest correlations
> :
* 
* 
* 
* 

In [ ]:
# fig, ax = plt.subplots(ncols=2, figsize=(10, 5))
# # sns.set(context='notebook', font_scale=1.35)

# # give a little more space between the 'suptitle' and figugre
# fig.suptitle("Distribution Comparison", y=1.02)
# # and between subplots
# plt.subplots_adjust(left=None, bottom=None, right=None, top=None,
#                 wspace=.3, hspace=5)

# for col in sqs.columns:
#     sns.distplot(sqs[col], ax=ax[0])
#     ax[0].set_title('current')
#     ax[0].set(xlabel='', yticklabels=[])
    
# for col in sqs_5000.columns:
#     sns.distplot(sqs_5000[col], ax=ax[1])
#     ax[1].set_title('dropping values above 5000 sqft')
#     ax[1].set(xlabel='', yticklabels=[])

# plt.legend((sqs.columns[0], sqs.columns[1], sqs.columns[2]), fontsize='x-small')
# plt.show()

It is interesting how the shapes of these distributions change with an adjustment the affects so few entries. We will move forward with these adjustments, for now.

* The `sqft_lot` variables are off the rails.

We may have some log transformations, in our future.

***
### Assumptions for Linear Regression

- Linearity between target and predictor variables
    - Previewed above, but the scales may be a bit unbalanced, yet
- Normality of model residuals
    - Verify after building a baseline model
- Homoscedasticity: equal variability of a dependent variable across the values of an independent variable
    - Review after normalizing data
- Absence of, or minimal multicollinearity
    - We will need to select the best and discard the rest from multicollinear variables 

In [ ]:
# custom_palette = sns.color_palette("Dark2")
# sns.pairplot(no_hots_df, x_vars=["bedrooms", "sqft_basement"], y_vars=["price"],
#              hue="waterfront", palette=custom_palette, height=5, aspect=.8, kind="reg");

## Pickle the model / current - state dataframe

Store the `*****` dataframe in a sub - directory of the repository as `****************`.

In [ ]:
# with open('data/*****.pickle', 'wb') as f:
#     # Pickling the '******' dataframe using the highest protocol available.
#     pickle.dump(******, f, pickle.HIGHEST_PROTOCOL)

## Questions Asked

## Questions Answered

## Next

* 

# Notes
***

In [ ]:
# import gc
# gc.get_count()
# gc.collect()
# gc.get_count()

## Dealing with Outliers

In [ ]:
# from scipy import stats

### Taking a look at how far out our outliers are

In [ ]:
# creating a function to identify columns in a dataframe
# with values more than 3 standard deviations from the mean
# def check_outliers(df, col):
#     std_min = df[col].mean() - 3*df[col].std()
#     std_max = df[col].mean() + 3*df[col].std()
#     if df[col].min() < std_min or df[col].max() > std_max:
#         print(f"\nValue in {col} exceeds +/- 3 standard deviations")
#         print(f"column max: {df[col].max()}, std.dev. max: {std_max}")
#         print(f"column mini: {df[col].min()}, std.dev. min: {std_min}")
#         print("-"*72)

# parking lot:

> 

In [ ]:
# no_hots_df.sqft_above.hist(alpha=.6, color='red')
# sqs.sqft_above.hist(alpha=.6, color='green')
# sqs_3000.sqft_above.hist(alpha=.6, color='blue')

# plt.show()

In [ ]:
# no_hots_df.sqft_lot.apply(np.log).hist(alpha=.5)
# no_hots_df.sqft_lot15.apply(np.log).hist(alpha=.3)
# plt.show()

In [ ]:

# plt.figure(figsize=(8,4))
# plt.hist(target, color='orange')
# plt.show()

In [ ]:
# scatter plot a map
# df.plot(kind="scatter", x="longitude", y="latitude", alpha=0.2)
# plt.savefig('map1.png')

In [ ]:
# color scatter plot map from least-to-most expensive
df.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4, figsize=(10,7),
    c="lastsoldprice", cmap=plt.get_cmap("jet"), colorbar=True,
    sharex=False)